## Notebook 11.1: *denovo* Genome assembly 

This week's assignment will take quite a bit longer than most others. This is not necessarily because it is more difficult, but because some of the computational steps take a while to run. So please be patient and start early.  


### Learning objectives: 

By the end of this notebook you will:

+ Be familiar with assembled genome file formats. 
+ Know where to find assembled genomes and their raw files online.
+ Be able to compute genome size from kmer counts of Illumina short reads. 
+ Have experience running Illumina, and Illumina+PacBio hybrid assembly.  

### Assigned readings: 

**Read these papers carefully, we will discuss in class:**

+ Li, Fay-Wei, and Alex Harkess. “A Guide to Sequence Your Favorite Plant Genomes.” Applications in Plant Sciences 6, no. 3 (n.d.): e1030. https://doi.org/10.1002/aps3.1030.



+ Sumpter, Nicholas, Margi Butler, and Russell Poulter. “Single-Phase PacBio De Novo Assembly of the Genome of the Chytrid Fungus Batrachochytrium Dendrobatidis, a Pathogen of Amphibia.” Microbiology Resource Announcements 7, no. 21 (November 2018). https://doi.org/10.1128/MRA.01348-18.



**Please skim the paper below, you do not need to read it in detail:** 
+ Giordano, Francesca, Louise Aigrain, Michael A. Quail, Paul Coupland, James K. Bonfield, Robert M. Davies, German Tischler, et al. “De Novo Yeast Genome Assemblies from MinION, PacBio and MiSeq Platforms.” Scientific Reports 7, no. 1 (June 21, 2017): 3935. https://doi.org/10.1038/s41598-017-03996-z.




### Software used in this notebook
We will use three Python packages and also a number of command-line programs that have been installed using the conda package manager. The commands to install the software are shown commented out below. You do not need to run the installation commands since the software is already installed for you. 

In [75]:
import toyplot
import numpy as np
import pandas as pd

In [76]:
# conda install numpy pandas
# conda install eaton-lab::toytree   
# conda install bioconda::cutadapt      # read adapter/quality trimmer
# conda install bioconda::jellyfish     # kmer counting
# conda install bioconda::spades        # debruijn graph assembler
# conda install bioconda::minimap2      # short/long read mapper/aligner
# conda install bioconda::miniasm       # OLC-based de novo assembler
# conda install bioconda::racon         # polisher: consensus calls on long read assembly   

https://yiweiniu.github.io/blog/2018/03/Genome-assembly-pipeline-miniasm-Racon/

### Computational resources for genome assembly

Genome assembly is very computationally intensive. When we talk about genome assembly we're typically referring to *de novo* genome assembly -- the process of assembling a genome without any form of *a priori* guide. This stands in contrast to reference-guided assembly methods, which are used to assemble a genome by mapping reads to an existing close relative's genome. The *de novo* process is incredibly more demanding both in terms of the amount of input data, and computational power that is required. 

In particular, *de novo* assembly requires enormous amounts of RAM (i.e., accessible memory). This type of resource is not easily shared across computing nodes of a cluster, and so instead HPC systems typically contain a few special nodes, designated *high-memory* nodes, that are  best for this process. As an example, even small bacterial genomes will typically require 8GB or more of RAM, and human genomes require 100-500Gb, depending on the software being used. Much larger genomes obviously require even more. High memory nodes often have 1Tb of RAM or more. 

To keep things simple for these exercises we will work with relatively small genomes and smallish numbers of sequenced reads. We will focus on the assembly of a Eukaryotic genome *Batrachochytrium dendrobatidis*, a Chytrid fungus that is an ecologically important pathogen of amphibians.   

## The *Bd* genome
We will assemble the *Bd* chytrid fungus genome from the assigned Sumpter et al. (2018) paper. I selected this paper because it provides a very concise report of a genome assembly and also where to find the associated data.

The first draft genome assembly of *Bd* genome was sequenced by the US DOE Joint Genome Institute using Sanger sequencing to 8.74X coverage in 2011. That assembly is in 127 scaffolds (N50 scaffold size of 1.48Mb) and 510 contigs (N50 contig size 318K) (https://www.ncbi.nlm.nih.gov/assembly/GCF_000203795.1). In your assigned p aper by Sumpter et al. a new draft genome is assembled using PacBio data (and some additional scaffolding methods). Our exercise in this notebook will be to re-assemble this data set using their data, and to explore alternative assembly methods. 


<div class="alert alert-success">
    <b>[1] Question:</b> 
    What are the assembly statistics for the initial draft genome assembled by Sumpter et al. using PacBio data (before they do additional scaffolding)? Answer in markdown below.  
</div>

In [8]:
#answer here:   


## Download sequence data (.fastq) files

The paper by Sumpter et al. includes a section called *Data availability* describing the accession number under which the data have been archived. This is typical for any genome paper. The genome assembly is deposited under the accession ID `QUAD00000000`, with the particular *version* of the genome assembled in this paper given the ID QUAD01000000. Future assemblies of the same data, perhaps using better software, could be uploaded and associated with this project, but given new accession IDs. This accession identifier can be used to query the assembly and retrieve the contigs which are labeled QUAD01000001-QUAD01000063. Follow the link in the paper to the genome project page [https://www.ncbi.nlm.nih.gov/nuccore/QUAD00000000](https://www.ncbi.nlm.nih.gov/nuccore/QUAD00000000). There you can find additional links to the Project accession, or the BioSample accession, all of which can tell you more about the project and how to access the data. As described in the Data Availability section of the paper, there were 10 other strains of Bd also sequenced in this study (using Illumina short read data) which are also linked under the same Project ID (PRJNA483086), but are not linked under the same Genome Assembly (QUAD00000000). By following links on the project page we can find pages for the sequenced individuals (SRXs) and sequencing runs (SRRs). I know, the accession IDs are hard to keep straight. These last IDs link to the actual sequence data files. For example, this page has details about the PacBio data we will be assembling: [https://www.ncbi.nlm.nih.gov/sra/SRX4676189[accn]](https://www.ncbi.nlm.nih.gov/sra/SRX4676189[accn]). 

Large fastq sequence data files are stored online in a weird format called `.sra` which includes a lot of connected metadata. This can be a pain when you just want to access the fastq files. We have installed a suite of command-line tools called the `sra-tools`, that allow you to query these databases using accession IDs, download the sequence data, and convert it into the proper format all in one command. From these tools the only one we really need is called  `fastq-dump`. The commands below will download the PacBio data for Bd, and Illumina data for one of the other strains included in the study which was shown to be identical to the one sequenced with PacBio. This download takes about 15 minutes, so get get a cup of coffee while it runs. The PacBio data is all single reads, whereas the Illumina data is paired-end reads (separate R1 and R2 files). We will use `head` to subsample both down to smaller numbers of reads for this exercise.

In [ ]:
%%bash

# Download Bd RTP6 PacBio reads
fastq-dump SRR7825134

# subsample PacBio data to 100K reads and rm full size file
head -n 400000 SRR7825134.fastq > SRR7825134.100K.fastq
rm SRR7825134.fastq

# Download Bd RTP5 Illumina HiSeq 2000 paired-end reads
fastq-dump SRR7825135 --split-files

# subsample Illumina data to 500K reads each and rm full sized files
head -n 2000000 SRR7825135_1.fastq > SRR7825135_1.500K.fastq
head -n 2000000 SRR7825135_2.fastq > SRR7825135_2.500K.fastq
rm SRR7825135_1.fastq SRR7825135_2.fastq

### The fastq data files
You have seen fastq data files before, and the PacBio data is no different from Illumina fastq data files, except the reads are just longer. We'll return to the PacBio reads in the next notebook, but for now we will focus on the Illumina short read data. How good of an assembly of the *Bd* genome can we get from short read data alone? 

<div class="alert alert-success">
    <b>[2] Question:</b> 
    How many sequenced reads are in each of the fastq data files (both downloaded and subsampled ones)? Use any method (bash or Python) to compute and print the answer in the cell below. 
</div>

In [13]:
# your answer here

### Trim reads for quality and adapters
This is an important step for any genome assembly. The cleaner your reads are the better and faster the genome will assemble. The command below uses the program `cutadapt` to trim Illumina adapters from the reads and trim from the 3' end when quality falls below a set threshold based on the quality scores in the fastq data files. 



In [55]:
%%bash

# -q: trim 3' when quality < 10
# -a: trim adapters from R1
# -A: trim adapters from R2
# -o: output R1 filename
# -p: output R2 filename

cutadapt \
    -q 10 \
    -a AGATCGGAAGAGC -A AGATCGGAAGAGC \
    -o SRR7825135_1.500K.trim.fastq \
    -p SRR7825135_2.500K.trim.fastq \
    SRR7825135_1.500K.fastq \
    SRR7825135_1.500K.fastq \
    --quiet

cutadapt version 1.12
Copyright (C) 2010-2016 Marcel Martin <marcel.martin@scilifelab.se>

cutadapt removes adapter sequences from high-throughput sequencing reads.

Usage:
    cutadapt -a ADAPTER [options] [-o output.fastq] input.fastq

For paired-end reads:
    cutadapt -a ADAPT1 -A ADAPT2 [options] -o out1.fastq -p out2.fastq in1.fastq in2.fastq

Replace "ADAPTER" with the actual sequence of your 3' adapter. IUPAC wildcard
characters are supported. The reverse complement is *not* automatically
searched. All reads from input.fastq will be written to output.fastq with the
adapter sequence removed. Adapter matching is error-tolerant. Multiple adapter
sequences can be given (use further -a options), but only the best-matching
adapter will be removed.

Input may also be in FASTA format. Compressed input and output is supported and
auto-detected from the file name (.gz, .xz, .bz2). Use the file name '-' for
standard input/output. Without the -o option, output is sent to standard output.



### Genome size and complexity
Your assigned reading by Li and Harkess titled "A guide to sequence your favorite plant genomes" describes some particular difficulties of assembling plant genomes, but most of these can be generalized to other organisms with difficult to assemble genomes as well. They make the important point that before starting any genome sequencing project it is important to learn as much as you can about your organism, as this should inform the type of approach that you take, and your expectations about the result you will get. Similarly, one of the first things you can do when you get some initial shotgun sequenced Illumina data is to investigate some of these same questions: How big is the genome, how variable, and what ploidy (has it experienced genome duplications)? It turns out we can learn answers to these questions by examining the distribution of kmer counts in the raw read data itself, without even having to map, align, or assemble it. 

<div class="alert alert-success">
    <b>[3] Question:</b> 
    In Figure 2 in Li and Harkess what characteristics of the kmer distribution plots are informative about heterozygosity and repetitiveness? Why does one distribution have one hump while the other has two? Answer in Markdown below.     
</div>

In [56]:
# answer here:

### Jellyfish: kmer counting tool
The [jellyfish](https://github.com/gmarcais/Jellyfish/tree/master/doc) program is a tool used to efficiently count kmers from a genome fasta file or fastq sequenced read files. It works much faster than the Python code we wrote last week to find all kmers in a sequence and is quite easy to use. It turns out we can learn a lot from examining the distribution of kmer counts. The quote below is from the Jellyfish paper by the authors, Marçais and Kingford (2011): 
    
<blockquote>

Given a string S, we are often interested in counting the number of occurrences in S of every substring of length k. These length-k substrings are called k-mers and the problem of determining the number of their occurrences is called k-mer counting.

Counting the k-mers in a DNA sequence is an important step in many applications. For example, genome assemblers using the overlap-layout-consensus paradigm, such as the Celera (Miller et al., 2008; Myers et al., 2000) and Arachne (Jaffe et al., 2003) assemblers, use k-mers shared by reads as seeds to find overlaps. Statistics on the number of occurrences of each k-mer are first computed and used to filter out which k-mers are used as seeds. Such k-mer count statistics are also used to estimate the genome size: if a large fraction of k-mers occur c times, we can estimate the sequencing coverage to be approximately c and derive an estimate of the genome size from c and the total length of the reads. In addition, in most short-read assembly projects, errors are corrected in the sequencing reads to improve the quality of the final assembly. For example, Kelley et al. (2010) use k-mer frequencies to assess the likelihood that a misalignment between reads is a sequencing error or a genuine difference in sequence. A third application is the detection of repeated sequences, such as transposons, which play an important biological role. De novo repeat annotation techniques find candidate regions based on k-mer frequencies (Campagna et al., 2005; Healy et al., 2003; Kurtz et al., 2008; Lefebvre et al., 2003). The counts of k-mers are also used to seed fast multiple sequence alignment (Edgar, 2004). Finally, k-mer distributions can produce new biological insights directly. Sindi et al. (2008) used k-mers frequencies with large k (20 ≤ k ≤ 100) to study the mechanisms of sequence duplication in genomes.
</blockquote>

Running jellyfish on the raw data files produces a `.jf` formatted file with information about the kmers in the data. On the full Illumina data files this will take about 5 minutes to run. We can then extract a histogram from this file using the `histo` function from jellyfish, which  returns the histogram as a text table showing the number of kmers that occurred N times. You can see many occurred very few times or only once, then it decreases before beginning to increase again. We run jellyfish on the full size data files because kmer counting requires ~30X or more to show good peaks. 

In [14]:
%%bash

# call jellyfish on short-read fastq files
# -m kmer size
# -s genome size estimate
# -t number of threads to use
# -C switch to count 'canonical kmers' (reverse complement)
# -o output location
# <(...) this decompresses the file as it is passed to jellyfish

# run jellyfish counting for k=21mer
jellyfish count \
    -m 21 -s 100M -t 10 -C \
    -o 21mer.jf \
    SRR7825135_1.fastq \
    SRR7825135_2.fastq
    
# write histogram of kmer counts to a file 
jellyfish histo 21mer.jf > 21mer.hist.csv

# print first 20 lines from histogram file to the screen
head -n 20 21mer.hist.csv


Let's now use Python to load the kmer count data with the Pandas library and plot it with the Toyplot library. We can see clearly that there is not best peak around at around 55X coverage or so. We will assume that this peak represents a single copy of the genome, and that any other peaks represent heterozygosity, errors, repetitions, and other such noise. So the density of our single-copy peak stretches from about 30 to 80. 

In [69]:
# load the histogram data as a table
df = pd.read_csv("../21mer.hist.csv", sep=" ", names=["bins", "counts"])
toyplot.fill(
    df.counts[2:200], 
    width=500, 
    height=300,
    ylabel="number of unique kmers", 
    xlabel="kmer coverage depth",
);

<polygon points="50.0,250.0 52.0,250.0 54.0,250.0 56.0,250.0 58.0,250.0 60.0,250.0 62.0,250.0 64.0,250.0 66.0,250.0 68.0,250.0 70.0,250.0 72.0,250.0 74.0,250.0 76.0,250.0 78.0,250.0 80.0,250.0 82.0,250.0 84.0,250.0 86.0,250.0 88.0,250.0 90.0,250.0 92.0,250.0 94.0,250.0 96.0,250.0 98.0,250.0 100.0,250.0 102.0,250.0 104.0,250.0 106.0,250.0 108.0,250.0 110.0,250.0 112.0,250.0 114.0,250.0 116.0,250.0 118.0,250.0 120.0,250.0 122.0,250.0 124.0,250.0 126.0,250.0 128.0,250.0 130.0,250.0 132.0,250.0 134.0,250.0 136.0,250.0 138.0,250.0 140.0,250.0 142.0,250.0 144.0,250.0 146.0,250.0 148.0,250.0 150.0,250.0 152.0,250.0 154.0,250.0 156.0,250.0 158.0,250.0 160.0,250.0 162.0,250.0 164.0,250.0 166.0,250.0 168.0,250.0 170.0,250.0 172.0,250.0 174.0,250.0 176.0,250.0 178.0,250.0 180.0,250.0 182.0,250.0 184.0,250.0 186.0,250.0 188.0,250.0 190.0,250.0 192.0,250.0 194.0,250.0 196.0,250.0 198.0,250.0 200.0,250.0 202.0,250.0 204.0,250.0 206.0,250.0 208.0,250.0 210.0,250.0 212.0,250.0 214.0,250.0 216.0,250.0 218.0,250.0 220.0,250.0 222.0,250.0 224.0,250.0 226.0,250.0 228.0,250.0 230.0,250.0 232.0,250.0 234.0,250.0 236.0,250.0 238.0,250.0 240.0,250.0 242.0,250.0 244.0,250.0 246.0,250.0 248.0,250.0 250.0,250.0 252.0,250.0 254.0,250.0 256.0,250.0 258.0,250.0 260.0,250.0 262.0,250.0 264.0,250.0 266.0,250.0 268.0,250.0 270.0,250.0 272.0,250.0 274.0,250.0 276.0,250.0 278.0,250.0 280.0,250.0 282.0,250.0 284.0,250.0 286.0,250.0 288.0,250.0 290.0,250.0 292.0,250.0 294.0,250.0 296.0,250.0 298.0,250.0 300.0,250.0 302.0,250.0 304.0,250.0 306.0,250.0 308.0,250.0 310.0,250.0 312.0,250.0 314.0,250.0 316.0,250.0 318.0,250.0 320.0,250.0 322.0,250.0 324.0,250.0 326.0,250.0 328.0,250.0 330.0,250.0 332.0,250.0 334.0,250.0 336.0,250.0 338.0,250.0 340.0,250.0 342.0,250.0 344.0,250.0 346.0,250.0 348.0,250.0 350.0,250.0 352.0,250.0 354.0,250.0 356.0,250.0 358.0,250.0 360.0,250.0 362.0,250.0 364.0,250.0 366.0,250.0 368.0,250.0 370.0,250.0 372.0,250.0 374.0,250.0 376.0,250.0 378.0,250.0 380.0,250.0 382.0,250.0 384.0,250.0 386.0,250.0 388.0,250.0 390.0,250.0 392.0,250.0 394.0,250.0 396.0,250.0 398.0,250.0 400.0,250.0 402.0,250.0 404.0,250.0 406.0,250.0 408.0,250.0 410.0,250.0 412.0,250.0 414.0,250.0 416.0,250.0 418.0,250.0 420.0,250.0 422.0,250.0 424.0,250.0 426.0,250.0 428.0,250.0 430.0,250.0 432.0,250.0 434.0,250.0 436.0,250.0 438.0,250.0 440.0,250.0 442.0,250.0 444.0,250.0 444.0,249.21386666666666 442.0,249.21386666666666 440.0,249.1829333333333 438.0,249.17226666666667 436.0,249.10986666666665 434.0,249.0941333333333 432.0,249.09333333333333 430.0,249.03786666666667 428.0,248.99786666666665 426.0,248.96266666666665 424.0,248.96266666666665 422.0,248.94213333333332 420.0,248.89333333333335 418.0,248.86880000000002 416.0,248.82026666666667 414.0,248.79973333333336 412.0,248.77413333333334 410.0,248.7165333333333 408.0,248.72666666666666 406.0,248.67946666666666 404.0,248.6288 402.0,248.66479999999999 400.0,248.61306666666667 398.0,248.58800000000002 396.0,248.53146666666666 394.0,248.57706666666667 392.0,248.57279999999997 390.0,248.61306666666667 388.0,248.55173333333335 386.0,248.4936 384.0,248.55253333333332 382.0,248.53573333333333 380.0,248.51386666666664 378.0,248.52559999999997 376.0,248.528 374.0,248.54959999999997 372.0,248.57146666666668 370.0,248.59893333333335 368.0,248.60266666666666 366.0,248.5829333333333 364.0,248.57706666666667 362.0,248.57173333333333 360.0,248.55226666666667 358.0,248.5608 356.0,248.4976 354.0,248.50159999999997 352.0,248.4864 350.0,248.48773333333332 348.0,248.5045333333333 346.0,248.4912 344.0,248.50799999999998 342.0,248.50719999999998 340.0,248.48373333333333 338.0,248.48613333333333 336.0,248.43573333333333 334.0,248.4 332.0,248.39679999999998 330.0,248.36 328.0,248.34133333333332 326.0,248.28346666666667 324.0,248.21866666666668 322.0,248.25146666666666 320.0,248.2544 318.0,248.21866666666668 316.0,248.11573333333334 314.0,248.02986666666666 312.0,247.99093333333332 310.0,247.9304 308.0,247.83946666666668 306.0,247.7250666666667 304.

### Genome size estimation
We can estimate the genome size by taking the total number of kmers (multiple the coverage times the number of unique kmers in each bin) and dividing this by the mean coverage (peak position). We get an estimate of ~30Mb, which is close to the current estimated value of the *Bd* genome of 24Mb.  

In [74]:
# calculate genome size estimate
np.sum(df.counts[2:] * df.bins[2:]) / 55

30522542

### *de novo* assembly methods

There are many programs available today for genome assembly. They vary in terms of requirements of the input data, their speed, RAM requirements, and the resulting accuracy of the assemblies. For some types of data, or sizes of genomes some tools will work better than others. Because *de novo* assembly is such a time consuming process it is difficult to compare many tools and so people often just use a program that others suggest. It's worth reading a few bioinformatics articles that compare assembly tools to learn more about which one might be best for your project. You optional assigned reading by Giordano et al. is one such paper. It compares several assembly tools on a variety of data sets. The tool we will use below, [spades](http://cab.spbu.ru/software/spades/), has been popular for years, and continually updated to keep pace with advancing sequencing technologies. It is a *de Bruijn* graph based assembly method that is best for short-read assembly, but which recently added support for *hybrid* assemblies that combine short-read contig assembly with scaffolding by using long-reads. We will try these approaches here. 

Because this genome and dataset are relatively small the spades assembly should complete in about 5 minutes or less. 

### *de novo* assembly with spades: Illumina only

In [ ]:
%%bash

spades.py \
    --only-assembler \
    -o assembly_spades/ \
    -1 SRR7825135_1.500K.trim.fastq \
    -2 SRR7825135_2.500K.trim.fastq \
    -k 21 \
    -m 8
    

### *de novo* HYBRID assembly with spades: Illumina & PacBio

In [ ]:
spades.py \
    --only-assembler \
    -o assembly_spades/ \
    -1 SRR7825135_1.500K.trim.fastq \
    -2 SRR7825135_2.500K.trim.fastq \
    --pacbio SRR7825134.fastq \
    -k 21 \
    -m 8


In [77]:
#quast-download-busco

<div class="alert alert-success">
    <b>[4] Question:</b> 
    In Figure 2 in Li and Harkess what characteristics of the kmer distribution plots are informative about heterozygosity and repetitiveness? Why does one distribution have one hump while the other has two? Answer in Markdown below.     
</div>

### Overlap: get overlap of PacBio reads
Here we align all of the PacBio reads to each other. Remember this is the type of step that we avoid doing in a *de Bruijn* graph based assembly, which is typically employed on short reads. Instead, in an OLC assembly method we start with the all-by-all comparisons. For very large datasets this can be time consuming. New algorithms (e.g., `minimap2` have *dramatically* improved the speed of this step. 

In [ ]:
%%bash

# overlap: get overlap of PacBio reads
minimap2 -x ava-pb -t 2 SRR7825134.fastq SRR7825134.fastq | gzip -1 > SRR7825134.paf.gz


### Layout: assemble contigs from overlapping reads
The program `miniasm` is an OLC assembler, it takes the raw data (.fastq) and the [pairwise alignment file (.paf)](https://github.com/lh3/miniasm/blob/master/PAF.md) with mapping information and constructs contigs based on the overlap among reads. An output is produced in [graphical fragement assembly format (.gfa)](https://github.com/GFA-spec/GFA-spec). We then use the unix tool `awk` to extract the lines from the .gfa file that contain the sequence contigs and write them to a fasta file. 

In [ ]:
%%bash

# layout: 
miniasm -f SRR7825134.fastq SRR7825134.paf.gz > SRR7825134.gfa

# extract sequence data from PAF to fasta
awk '$1 ~/S/ {print ">"$2"\n"$3}' SRR7825134.gfa > SRR7825134.fasta


### Consensus: get consensus (GFA to fasta)

This step will map the raw reads (.fastq) to the new assembled contigs (.fasta) and save the results in a [pairwise alignment file (.paf)](https://github.com/lh3/miniasm/blob/master/PAF.md). The polishing tool `racon` takes then takes the assembled contigs, the alignment file, and the raw data all together and attempts to fill gaps and correct base calls by comparing raw reads where they map on assembled contigs. The mapping step takes only about a minute on this dataset, polishing takes about 15 minutes. 

In [ ]:
%%bash

# map raw reads to the long-read assembled contigs (fasta)
minimap2 -t 2 SRR7825134.fasta SRR7825134.fastq > SRR7825134.gfa1.paf

# polish/correct the assembly using the mapped short reads
racon -t 2 SRR7825134.fastq SRR7825134.gfa1.paf SRR7825134.fasta > SRR7825134.racon1.fasta


### Look at your final genome file
This is it. You assembled a genome completely using long-read PacBio data. 

### Computational resources for genome assembly

Genome assembly is a very computationally intensive process. In particular, it requires a large amount of RAM (accessible memory). This type of resource cannot be easily shared across computing nodes a cluster, and instead HPC systems typically contain a few special nodes that are designated *high-memory* nodes, which are often best used for genome assemblies. For example, even a small bacterial genome will typically require 8GB or more of RAM. Depending on the assembly software Human genomes typically require 100-500Gb of RAM. Much larger genomes obviously require even more. To make things simple for these exercises we are limiting our analyses to small genomes and smallish numbers of sequenced reads. 

### Question from last time: PacBio 'subreads'
http://seqanswers.com/forums/showthread.php?t=34790


Subreads are the individual sequence reads that are determined in real time from a template on the sequencer. These would correspond to the stretch of DNA between the adapter sequence.

CCS reads are the result of doing a consensus base calling from subreads that are all from the same template. If the template was short enough, then the polymerase will loop entirely from the beginning back to the beginning, sequencing the adapter before it starts on the template again. In this case, the same template piece of DNA is sequenced more than once, which means that the sequence data generated from each pass can be used to determine one single consensus sequence with higher base quality than the raw subreads. Thus, the quality distribution that you see is correct between the subreads and the CCS reads



### Estimate genome size from kmers distribution
The size parameter (given with -s) is an indication of the number k-mers that will be stored in the hash. For sequencing reads, this size should be the size of the genome plus the k-mers generated by sequencing errors. For example, if the error rate is e (e.g.Illumina reads, usually e~1%), with an estimated genome size of G and a coverage of c, the number of expected k-mers is G+Gcek.

In [37]:
%%bash

jellyfish stats jellyfish/11mer.jf

Unique:    6574
Distinct:  7086
Total:     7807
Max_count: 10


### Pilon: Genome assembly improvement software
This desription is based on the [Pilon documentation](https://github.com/broadinstitute/pilon/wiki), where you can find more details. Pilon is used to improve genome assemblies based on  how short reads map it. It is most often used when a genome assembly is based on long reads, which have higher error rates than short reads, and may introduce spurious structural variants. Pilon takes as input a FASTA genome file and one or more BAM files of reads aligned to the input FASTA genome. Pilon aims to improve the assembly by correcting the following types of variation: 

+ Single base differences
+ Small indels
+ Larger indel or block substitution events
+ Gap filling
+ Identification of local misassemblies, including optional opening of new gaps

Pilon then outputs a FASTA file containing an improved representation of the genome from the read data and an optional VCF file detailing variation seen between the read data and the input genome.